In [ ]:
import torchvision
from torchvision import transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import os
import torch
import torch.nn as nn
from torchvision.models import resnet18
import torch.optim as optim
from sklearn.metrics import classification_report, accuracy_score, f1_score, precision_score, recall_score

In [ ]:
base_dir = os.getcwd()

In [20]:
dataset_path = os.path.join(base_dir, 'data_images')

In [21]:
# Mean e std per immagini RGB normalizzate su [-1, 1]
mean = [0.5, 0.5, 0.5]
std = [0.5, 0.5, 0.5]

train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.1, contrast=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

val_test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

# per fare data augmentation utiliziammo transforms.Compose di torchvision, facciamo data augmentation solo ai set di train

In [22]:
train_dataset = datasets.ImageFolder(root=os.path.join(dataset_path, "train"), transform=train_transform)
val_dataset = datasets.ImageFolder(root=os.path.join(dataset_path, "valid"), transform=val_test_transform)
test_dataset = datasets.ImageFolder(root=os.path.join(dataset_path, "test"), transform=val_test_transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Classi leggibili (opzionale)
pretty_classes = ['Adenocarcinoma', 'Adgelcarcinoma', 'Squamosgelcarcinoma', 'Noncancer']

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = resnet18(pretrained=True) # trasnfer learning

# congela i layer convoluzionali per fare fine-tuning solo sull'ultimo layer
for param in model.parameters():
    param.requires_grad = False

# Sblocca solo l'ultimo FC Layer
num_features = model.fc.in_features # ultimo layer che mappa le classi
model.fc = nn.Linear(num_features, 4)  # 4 classi

model = model.to(device)


c:\Users\noemi\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\noemi\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\noemi/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:11<00:00, 3.92MB/s]


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

In [25]:
def train_model(model, train_loader, val_loader, epochs=10):
    for epoch in range(epochs):
        model.train()
        train_loss = 0
        correct = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()

        train_acc = correct / len(train_loader.dataset)
        scheduler.step()

        # Valutazione
        model.eval()
        val_loss = 0
        correct = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                preds = outputs.argmax(dim=1)
                correct += (preds == labels).sum().item()
        val_acc = correct / len(val_loader.dataset)

        print(f"Epoch {epoch+1}/{epochs} | Train Loss: {train_loss:.3f} | Train Acc: {train_acc:.3f} | Val Acc: {val_acc:.3f}")


In [26]:
train_model(model, train_loader, val_loader, epochs=10)

Epoch 1/10 | Train Loss: 25.605 | Train Acc: 0.423 | Val Acc: 0.375
Epoch 2/10 | Train Loss: 20.885 | Train Acc: 0.574 | Val Acc: 0.417
Epoch 3/10 | Train Loss: 18.333 | Train Acc: 0.630 | Val Acc: 0.431
Epoch 4/10 | Train Loss: 16.790 | Train Acc: 0.659 | Val Acc: 0.486
Epoch 5/10 | Train Loss: 16.184 | Train Acc: 0.682 | Val Acc: 0.556
Epoch 6/10 | Train Loss: 14.900 | Train Acc: 0.726 | Val Acc: 0.472
Epoch 7/10 | Train Loss: 14.450 | Train Acc: 0.715 | Val Acc: 0.597
Epoch 8/10 | Train Loss: 14.665 | Train Acc: 0.729 | Val Acc: 0.486
Epoch 9/10 | Train Loss: 14.809 | Train Acc: 0.698 | Val Acc: 0.583
Epoch 10/10 | Train Loss: 14.703 | Train Acc: 0.726 | Val Acc: 0.569


In [ ]:
def evaluate_metrics(model, loader, class_names):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = outputs.argmax(dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Accuracy
    acc = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average='macro')
    precision = precision_score(all_labels, all_preds, average='macro')
    recall = recall_score(all_labels, all_preds, average='macro')

    print(f"Accuracy : {acc:.4f}")
    print(f"F1 Score : {f1:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall   : {recall:.4f}")
    print("\nDetailed per-class metrics:")
    print(classification_report(all_labels, all_preds, target_names=class_names))


In [29]:
pretty_classes = ['Adenocarcinoma', 'Adgelcarcinoma', 'Squamosgelcarcinoma', 'Noncancer']
evaluate_metrics(model, test_loader, pretty_classes)


Accuracy : 0.6095
F1 Score : 0.5749
Precision: 0.6825
Recall   : 0.5783

Detailed per-class metrics:
                     precision    recall  f1-score   support

     Adenocarcinoma       0.52      0.84      0.64       120
     Adgelcarcinoma       0.67      0.16      0.25        51
Squamosgelcarcinoma       1.00      0.98      0.99        54
          Noncancer       0.55      0.33      0.41        90

           accuracy                           0.61       315
          macro avg       0.68      0.58      0.57       315
       weighted avg       0.63      0.61      0.57       315

